In [2]:
import torch
import os
import torchvision
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn as nn
import time
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter


device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print("Using device:", device)

In [3]:
def get_cifar100_data_loaders(download, shuffle=True, batch_size=128, num_workers=8):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
    ])

    train_dataset = datasets.CIFAR100('./datasets', train=True, download=download,
                                      transform=transform_train)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,
                              num_workers=num_workers, shuffle=shuffle)

    test_dataset = datasets.CIFAR100('./datasets', train=False, download=download,
                                     transform=transform_test)

    test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                             num_workers=num_workers, shuffle=shuffle)
    
    return train_loader, test_loader

In [4]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [8]:
def train(model, train_loader, test_loader, device, epochs=200):
    current_time = time.strftime("%Y%m%d-%H%M%S")
    log_dir = os.path.join('./runs', f'LCP+{current_time}')
    
    writer = SummaryWriter(log_dir=log_dir)
    
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs")
        model = torch.nn.DataParallel(model)
    
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=8e-4)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    scheduler = None
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader), eta_min=1e-6)
    
    for epoch in tqdm(range(epochs), desc="Epochs", ncols=75):
        model.train()
        top1_train_accuracy = 0
        for counter, (x_batch, y_batch) in enumerate(train_loader):
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)

            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            top1 = accuracy(logits, y_batch, topk=(1,))
            top1_train_accuracy += top1[0]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        top1_train_accuracy /= (counter + 1)
        if scheduler is not None:
            scheduler.step()
        
        model.eval()
        top1_accuracy = 0
        top5_accuracy = 0
        
        with torch.no_grad():
            for counter, (x_batch, y_batch) in enumerate(test_loader):
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                logits = model(x_batch)

                top1, top5 = accuracy(logits, y_batch, topk=(1, 5))
                top1_accuracy += top1[0]
                top5_accuracy += top5[0]

        top1_accuracy /= (counter + 1)
        top5_accuracy /= (counter + 1)

        # 写入TensorBoard
        writer.add_scalar('Loss/train', loss.item(), epoch)
        writer.add_scalar('Accuracy/Top1 Train', top1_train_accuracy.item(), epoch)
        writer.add_scalar('Accuracy/Top1 Test', top1_accuracy.item(), epoch)
        writer.add_scalar('Accuracy/Top5 Test', top5_accuracy.item(), epoch)
        
        # 打印输出，控制到两位小数
        print(f"Epoch {epoch} Loss: {loss.item():.4f} Top1 train_acc: {top1_train_accuracy.item():.2f} Top1 test_acc: {top1_accuracy.item():.2f} Top5 test_acc: {top5_accuracy.item():.2f}")
    
    # 关闭writer
    writer.close()

### Linear Classification Protocal

In [16]:
# Load checkpoints

model = torchvision.models.resnet18(weights=None, num_classes=100).to(device)
# checkpoint = torch.load('./runs/resnet18-1.00-200epochs/checkpoint_0200.pth.tar', map_location=device)
# checkpoint = torch.load('./runs/resnet18-0.75-150epochs/checkpoint_0125.pth.tar', map_location=device)
# checkpoint = torch.load('./runs/resnet18-0.50-150epochs/checkpoint_0125.pth.tar', map_location=device)
checkpoint = torch.load('./runs/resnet18-0.25-150epochs/checkpoint_0125.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):
  if k.startswith('module.backbone.'):
    if k.startswith('module.backbone') and not k.startswith('module.backbone.fc'):
      # remove prefix
      state_dict[k[len("module.backbone."):]] = state_dict[k]
  del state_dict[k]
  
log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']


train_loader, test_loader = get_cifar100_data_loaders(download=True)

Files already downloaded and verified
Files already downloaded and verified


In [17]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [18]:
train(model, train_loader, test_loader, device, epochs=40)

Epochs:   2%|▊                              | 1/40 [00:06<04:21,  6.70s/it]

Epoch 0 Loss: 3.9783 Top1 train_acc: 8.69 Top1 test_acc: 12.49 Top5 test_acc: 32.78


Epochs:   5%|█▌                             | 2/40 [00:13<04:13,  6.66s/it]

Epoch 1 Loss: 3.6910 Top1 train_acc: 12.45 Top1 test_acc: 14.35 Top5 test_acc: 36.04


Epochs:   8%|██▎                            | 3/40 [00:20<04:09,  6.74s/it]

...

Epoch 37 Loss: 3.6313 Top1 train_acc: 18.35 Top1 test_acc: 19.47 Top5 test_acc: 43.44


Epochs:  98%|█████████████████████████████▎| 39/40 [04:22<00:06,  6.80s/it]

Epoch 38 Loss: 3.6181 Top1 train_acc: 18.53 Top1 test_acc: 20.11 Top5 test_acc: 43.70


Epochs: 100%|██████████████████████████████| 40/40 [04:29<00:00,  6.73s/it]

Epoch 39 Loss: 3.4930 Top1 train_acc: 18.33 Top1 test_acc: 19.45 Top5 test_acc: 43.52


### Load Pretrained Model

In [20]:
model = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights).to(device)
model.fc = nn.Linear(512, 100)
train(model, train_loader, test_loader, device, epochs=40)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:04<00:00, 9.93MB/s]
Epochs:   2%|▊                              | 1/40 [00:07<04:54,  7.55s/it]

Epoch 0 Loss: 2.2843 Top1 train_acc: 30.24 Top1 test_acc: 41.27 Top5 test_acc: 74.08


Epochs:   5%|█▌                             | 2/40 [00:15<04:48,  7.60s/it]

Epoch 1 Loss: 2.1799 Top1 train_acc: 44.19 Top1 test_acc: 46.88 Top5 test_acc: 77.82


Epochs:   8%|██▎                            | 3/40 [00:22<04:35,  7.46s/it]

...

Epoch 37 Loss: 1.0023 Top1 train_acc: 73.97 Top1 test_acc: 58.22 Top5 test_acc: 84.77


Epochs:  98%|█████████████████████████████▎| 39/40 [04:48<00:07,  7.38s/it]

Epoch 38 Loss: 0.9445 Top1 train_acc: 74.41 Top1 test_acc: 57.42 Top5 test_acc: 83.90


Epochs: 100%|██████████████████████████████| 40/40 [04:55<00:00,  7.39s/it]

Epoch 39 Loss: 1.0252 Top1 train_acc: 74.27 Top1 test_acc: 57.83 Top5 test_acc: 84.63


### Training from scratch

In [21]:
model = torchvision.models.resnet18(weights=None, num_classes=100).to(device)
train(model, train_loader, test_loader, device, epochs=40)

Epochs:   2%|▊                              | 1/40 [00:07<04:44,  7.28s/it]

Epoch 0 Loss: 3.4538 Top1 train_acc: 13.61 Top1 test_acc: 19.57 Top5 test_acc: 48.41


Epochs:   5%|█▌                             | 2/40 [00:14<04:39,  7.36s/it]

Epoch 1 Loss: 2.9726 Top1 train_acc: 23.07 Top1 test_acc: 27.66 Top5 test_acc: 58.42


Epochs:   8%|██▎                            | 3/40 [00:22<04:31,  7.34s/it]

...

Epoch 37 Loss: 1.1642 Top1 train_acc: 66.89 Top1 test_acc: 53.28 Top5 test_acc: 81.82


Epochs:  98%|█████████████████████████████▎| 39/40 [04:49<00:07,  7.74s/it]

Epoch 38 Loss: 0.8796 Top1 train_acc: 67.15 Top1 test_acc: 53.57 Top5 test_acc: 80.77


Epochs: 100%|██████████████████████████████| 40/40 [04:57<00:00,  7.44s/it]

Epoch 39 Loss: 1.2204 Top1 train_acc: 67.98 Top1 test_acc: 53.99 Top5 test_acc: 81.28
